In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [2]:
import glob

company_source_excel_files = glob.glob("*-companies.xlsx")
company_source_excel_files

['Outscraper-20251130-champaign-urbana-apartment-companies.xlsx',
 'Outscraper-20251130-penn-state-apartment-companies.xlsx',
 'Outscraper-20251201-byu-provo-apartment-companies.xlsx',
 'Outscraper-20251201-purdue-west-lafayette-apartment-companies.xlsx']

In [3]:
df_combined = None

for file in company_source_excel_files:
    df_part = pd.read_excel(file)

    print(f"Shape of {file} dataframe: {df_part.shape}")

    if df_combined is None:
        df_combined = df_part
    else:
        df_combined = pd.concat([df_combined, df_part], ignore_index=True)

print(f"Combined dataframe shape: {df_combined.shape}")

Shape of Outscraper-20251130-champaign-urbana-apartment-companies.xlsx dataframe: (323, 66)
Shape of Outscraper-20251130-penn-state-apartment-companies.xlsx dataframe: (328, 66)
Shape of Outscraper-20251201-byu-provo-apartment-companies.xlsx dataframe: (500, 66)
Shape of Outscraper-20251201-purdue-west-lafayette-apartment-companies.xlsx dataframe: (396, 66)
Combined dataframe shape: (1547, 66)


In [4]:
df_combined.columns

Index(['query', 'name', 'name_for_emails', 'site', 'subtypes', 'category',
       'type', 'phone', 'full_address', 'borough', 'street', 'city',
       'postal_code', 'state', 'us_state', 'country', 'country_code',
       'latitude', 'longitude', 'h3', 'time_zone', 'plus_code', 'area_service',
       'rating', 'reviews', 'reviews_link', 'reviews_tags',
       'reviews_per_score', 'reviews_per_score_1', 'reviews_per_score_2',
       'reviews_per_score_3', 'reviews_per_score_4', 'reviews_per_score_5',
       'photos_count', 'photo', 'street_view', 'located_in', 'working_hours',
       'working_hours_csv_compatible', 'working_hours_old_format',
       'other_hours', 'popular_times', 'business_status', 'about', 'range',
       'prices', 'posts', 'logo', 'description', 'typical_time_spent',
       'verified', 'owner_id', 'owner_title', 'owner_link',
       'reservation_links', 'booking_appointment_link', 'menu_link',
       'order_links', 'location_link', 'location_reviews_link', 'place_id',

In [5]:
df_combined["category"].value_counts()

category
Apartment building                      539
Apartment complex                       318
Property management company             170
Real estate agency                       96
Apartment rental agency                  85
Student housing center                   78
Real estate agent                        65
Condominium complex                      23
hotels                                   16
Student dormitory                        15
Furnished apartment building             14
University                               12
Commercial real estate agency            12
College                                   7
Real estate rental agency                 6
Townhouse complex                         6
Real estate developer                     5
Vacation home rental agency               5
Real estate consultant                    5
Edificio de apartamentos                  5
Short term apartment rental agency        4
Car rental agency                         4
Housing complex        

In [6]:
valid_categories = [
    "Apartment complex",
    "Property management company",
    "Apartment building",
    "Apartment rental agency",
    "Furnished apartment building",
    "Vacation home rental agency",
    "Housing complex",
    "Housing cooperative",
    "Apartments",
    "Leasing service",
    "Condominium rental agency",
]

In [7]:
valid_states = ["Illinois", "Utah", "Pennsylvania", "Indiana"]

In [8]:
df = df_combined.copy()

df = df[
    (df["reviews"] >= 10)
    & (df["business_status"] == "OPERATIONAL")
    & (df["category"].isin(valid_categories))
    & (df["us_state"].isin(valid_states))
]

df = df[~((df["us_state"] == "Utah") & (~df["city"].isin(["Provo", "Orem"])))]
df = df[~((df["us_state"] == "Pennsylvania") & (~df["city"].isin(["State College"])))]
df = df[~((df["us_state"] == "Illinois") & (~df["city"].isin(["Champaign", "Urbana"])))]
df = df[
    ~(
        (df["us_state"] == "Indiana")
        & (~df["city"].isin(["West Lafayette", "Lafayette"]))
    )
]

columns_to_drop = [
    "full_address",
    "phone",
    "query",
    "name_for_emails",
    "state",
    "country",
    "country_code",
    "h3",
    "plus_code",
    "area_service",
    "time_zone",
    "reviews_link",
    "reviews_tags",
    "reviews_per_score",
    "located_in",
    "working_hours",
    "working_hours_csv_compatible",
    "working_hours_old_format",
    "other_hours",
    "popular_times",
    "street_view",
    "business_status",
    "typical_time_spent",
    "photo",
    "description",
    "reservation_links",
    "booking_appointment_link",
    "menu_link",
    "order_links",
    "about",
    "posts",
    "range",
    "logo",
    "prices",
    "kgmid",
    "reviews_id",
    "located_google_id",
    "owner_id",
    "owner_title",
    "owner_link",
    "location_reviews_link",
    "cid",
]

nullable_int_columns = [
    "reviews",
    "reviews_per_score_1",
    "reviews_per_score_2",
    "reviews_per_score_3",
    "reviews_per_score_4",
    "reviews_per_score_5",
    "photos_count",
]

for col in nullable_int_columns:
    df[col] = df[col].astype("Int64")

df.drop(columns=columns_to_drop, inplace=True)

display(df.head(1))
print(df.shape)

,name,site,subtypes,category,type,borough,street,city,postal_code,us_state,latitude,longitude,rating,reviews,reviews_per_score_1,reviews_per_score_2,reviews_per_score_3,reviews_per_score_4,reviews_per_score_5,photos_count,verified,location_link,place_id,google_id
0,JSJ Property Management,http://www.jsjmanagement.com/,Property management company,Property management company,Property management company,NaN,302 W Springfield Ave,Champaign,61820.0,Illinois,40.112886,-88.247301,4.0,607,132,11,11,18,435,19,True,https://www.google.com/maps/place/JSJ+Property...,ChIJr-EbgDTXDIgRDpkDYnR9rP8,0x880cd734801be1af:0xffac7d746203990e


(407, 24)


In [9]:
df.to_csv("apartments-preprocessed.csv", index=False)